In [ ]:
import pandas as pd
from branca.element import Figure
import time
import random
import hashlib
from pathlib import Path
import requests
from tqdm import tqdm

## Carga de datos extraidos de la página del INSQUI

In [ ]:
df = pd.read_csv('../../data/raw/insqui/coordenadas_insqui_limpio.csv', header=0)

df.describe()

## Consulta variables climatológicas

In [ ]:
# Configuración inicial

OUT_DIR = Path("../../data/raw/nasa_power")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Últimos 10 años
START_YEAR = 2016
END_YEAR   = 2025

# Agrupación de coordenadas "idénticas"
DECIMALS = 4  

# NASA POWER API URL
NASA_URL = "https://power.larc.nasa.gov/api/temporal/daily/point"

# Variables climáticas enfocadas a hazard
PARAMETERS = ",".join([
    "T2M",                 # temperatura
    "RH2M",                # humedad
    "ALLSKY_SFC_SW_DWN",   # radiación
    "PRECTOTCORR",         # precipitación corregida
    "WS2M",                # viento
    "EVLAND"               # evapotranspiración
])

# Configuración de la API
API_CONFIG = {
    "community": "AG",
    "format": "JSON",
    "units": "metric",
    "time-standard": "UTC"
}

# Parámetros de reintento y espera entre solicitudes
SLEEP_RANGE = (1.2, 2.0)
MAX_RETRIES = 5
TIMEOUT_S   = 120

In [ ]:
# Funciones auxiliares

# Normalizar nombres de columnas
def _norm_cols(_df: pd.DataFrame) -> pd.DataFrame:
    _df = _df.copy()
    _df.columns = [c.strip().lower() for c in _df.columns]
    return _df

# Crear hash único para coordenadas redondeadas
def _make_hash(lat_r: float, lon_r: float, decimals: int) -> str:
    s = f"{lat_r:.{decimals}f},{lon_r:.{decimals}f}"
    return hashlib.md5(s.encode("utf-8")).hexdigest()[:12]

# Llamar a la API de NASA POWER con reintentos
def call_nasa(lat, lon, start, end):
    params = {
        "start": start,
        "end": end,
        "latitude": lat,
        "longitude": lon,
        "parameters": PARAMETERS,
        **API_CONFIG
    }

    last_err = None
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            r = requests.get(NASA_URL, params=params, timeout=TIMEOUT_S)

            # errores temporales comunes
            if r.status_code in (200, 422, 429):
                raise RuntimeError(f"HTTP {r.status_code}: {r.text[:200]}")

            r.raise_for_status()
            return r.json()

        except Exception as e:
            last_err = e
            backoff = min(30, (2 ** attempt) + random.uniform(0, 1.5))
            time.sleep(backoff)

    raise RuntimeError(f"Fallo tras {MAX_RETRIES} reintentos. Último error: {last_err}")

# Parsear JSON de respuesta de NASA POWER a DataFrame
def parse_json(data):
    param_block = data.get("properties", {}).get("parameter", {})
    if not param_block:
        return pd.DataFrame()

    any_var = next(iter(param_block.keys()))
    times = list(param_block.get(any_var, {}).keys())

    rows = []
    for t in times:
        row = {"date": t}
        for var in PARAMETERS.split(","):
            row[var] = param_block.get(var, {}).get(t, None)
        rows.append(row)

    out = pd.DataFrame(rows)
    out["date"] = pd.to_datetime(out["date"], errors="coerce")
    out = out.dropna(subset=["date"]).sort_values("date").reset_index(drop=True)

    # convertir numéricos y limpiar -999
    for var in PARAMETERS.split(","):
        out[var] = pd.to_numeric(out[var], errors="coerce")
        out.loc[out[var] <= -900, var] = pd.NA

    return out

In [12]:
# Preparación de df para llamadas API

# Normalizar nombres de columnas
df = _norm_cols(df)

# Añadir ID de fila
df = df.reset_index(drop=True)
df["row_id"] = df.index.astype(int)

# Crear coordenadas redondeadas y hash
df["lat_r"] = df["latitud"].astype(float).round(DECIMALS)
df["lon_r"] = df["longitud"].astype(float).round(DECIMALS)
df["coord_hash"] = df.apply(lambda r: _make_hash(r["lat_r"], r["lon_r"], DECIMALS), axis=1)

# Guardar índice de coordenadas
df[["row_id", "tipo_actividad", "latitud", "longitud", "lat_r", "lon_r", "coord_hash"]].to_csv(
    OUT_DIR / "coord_index.csv", index=False
)

# DataFrame de coordenadas únicas
df_coords_unique = (
    df[["coord_hash", "lat_r", "lon_r"]]
    .drop_duplicates("coord_hash")
    .sort_values("coord_hash")
    .reset_index(drop=True)
)

df_coords_unique

,coord_hash,lat_r,lon_r
0,0185cdf9e600,5.54,-73.35
1,021f18273d88,12.58,-81.70
2,071317421089,4.92,-74.02
3,0a88d5087728,6.28,-75.44
4,0d582b03d53e,3.23,-76.42
...,...,...,...
79,f109266adbd9,4.54,-75.68
80,f45a4eb79dd7,3.03,-76.41
81,f482b38c33e4,10.91,-74.79
82,f564773235f7,6.46,-75.56


In [17]:
# Llamadas a la API y guardado de resultados

for _, row in tqdm(df_coords_unique.iterrows(), total=len(df_coords_unique), desc="Coordenadas únicas"):
    uid = row["coord_hash"]
    lat = row["lat_r"]
    lon = row["lon_r"]

    out = OUT_DIR / f"{uid}_nasa_power.csv"

    if out.exists():
        print("ya existe:", uid)
        continue

    full_df = []

    for year in range(START_YEAR, END_YEAR + 1):
        start, end = f"{year}0101", f"{year}1231"

        try:
            j = call_nasa(lat, lon, start, end)
            df_year = parse_json(j)
            if not df_year.empty:
                full_df.append(df_year)

            time.sleep(random.uniform(*SLEEP_RANGE))

        except Exception as e:
            print(f"[WARN] coord={uid} year={year} error:", e)

    if full_df:
        final = pd.concat(full_df, ignore_index=True).sort_values("date").reset_index(drop=True)
        final.to_csv(out, index=False)

print("\n Listo:")
print(f"- Cache por coordenada en: {OUT_DIR}  (archivos *_nasa_power.csv)")
print(f"- Índice de mapeo filas -> coordenada: {OUT_DIR / 'coord_index.csv'}")

Coordenadas únicas: 100%|██████████| 84/84 [50:36<00:00, 36.15s/it]


 Listo:
- Cache por coordenada en: data/nasa_power  (archivos *_nasa_power.csv)
- Índice de mapeo filas -> coordenada: data/nasa_power/coord_index.csv
